In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import gc
import pandas as pd
import numpy as np
import torch
import datetime
from tqdm import tqdm_notebook
from torch import nn
from torchvision import transforms
from cnn_finetune import make_model

import model as M
import model2 as M2
import model3 as M3
import model4 as M4
from dataset import SegmentationDataset, SegmentationInferenceDataset
from data import *
from util import *

In [3]:
torch.cuda.is_available()
device = torch.device('cuda')

In [4]:
_, _, test_df = get_dfs()

In [5]:
test_dataset = SegmentationInferenceDataset(test_df, input_size=(128, 128), with_gt=False, with_raw_input=False, use_depth_channels=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

# First submission

In [7]:
gc.collect()
torch.cuda.empty_cache()

model = M.UNet(1, n_classes=1)
weight = torch.load(str(model_dir / 'unet_test_dict.model'))
model.load_state_dict(weight)
model = model.to(device)

In [16]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict(model, x, device=device)
        rles += [rle_encode(p.numpy()) for p in pred]

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


In [20]:
test_df['rle_mask'] = rles

In [28]:
submit(test_df, 'first_submission.csv')

# UNetRes submission

In [6]:
gc.collect()
torch.cuda.empty_cache()

model = M.UNetRes(1, n_classes=1)
weight = torch.load(str(model_dir / 'unet_resblock_dict.model'))
model.load_state_dict(weight)
model = model.to(device)
model.eval()

UNetRes(
  (inc): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (down1): down(
    (mpconv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): BasicResBlock(
        (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (downsample): Sequential(
          (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
    )
  )
  (down2): down(
    (mpconv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0,

In [7]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict(model, x, device=device)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res_submission.csv')

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


# UNetRes BCELoss SGD 3ch submission

In [11]:
gc.collect()
torch.cuda.empty_cache()

model = M.UNetRes(n_channels=3, n_classes=1)
weight = torch.load(str(model_dir / 'unet_res_bceloss_sgd_3ch_dict.model'))
model.load_state_dict(weight)
model = model.to(device)
model.eval()

UNetRes(
  (inc): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (down1): down(
    (mpconv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): BasicResBlock(
        (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (downsample): Sequential(
          (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
    )
  )
  (down2): down(
    (mpconv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0,

In [12]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict(model, x, device=device)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res_bceloss_sgd_3ch_submission.csv')

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


# UNetRes LovaszLoss SGD 3ch submission

In [6]:
gc.collect()
torch.cuda.empty_cache()

model = M.UNetRes(n_channels=3, n_classes=1)
weight = torch.load(str(model_dir / 'unet_res_lovaszloss_sgd_3ch_dict.model'))
model.load_state_dict(weight)
model = model.to(device)
model.eval()


UNetRes(
  (inc): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (down1): down(
    (mpconv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): BasicResBlock(
        (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (downsample): Sequential(
          (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
    )
  )
  (down2): down(
    (mpconv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0,

In [7]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict(model, x, device=device)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res_lovaszloss_sgd_3ch_submission.csv')

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


# Model2 UNetRes34HcAux

In [8]:
gc.collect()
torch.cuda.empty_cache()

model = M2.UNetRes34HcAux(n_classes=1)
weight = torch.load(str(model_dir / 'unet_res34_hcaux_dict.model'))
model.load_state_dict(weight)
model = model.to(device)
model.eval()

Loaded pretrained resnet weights


UNetRes34HcAux(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running

In [9]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict(model, x, device=device)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res34_hcaux_submission.csv')

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


In [10]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict(model, x, device=device, with_tta=True)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res34_hcaux_tta_submission.csv')

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


# Model2 UNetRes34HcAux lovaszloss ft

In [6]:
gc.collect()
torch.cuda.empty_cache()

model = M2.UNetRes34HcAux(n_classes=1)
weight = torch.load(str(model_dir / 'unet_res34_hcaux_lovaszloss_ft_dict.model'))
model.load_state_dict(weight)
model = model.to(device)
model.eval()

Loaded pretrained resnet weights


UNetRes34HcAux(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running

In [7]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict(model, x, device=device, with_tta=True)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res34_hcaux_lovaszloss_ft_tta_submission.csv')

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


# Model2 UNetRes34HcAux lovaszloss ft lr0.001

In [6]:
gc.collect()
torch.cuda.empty_cache()

model = M2.UNetRes34HcAux(n_classes=1)
weight = torch.load(str(model_dir / 'unet_res34_hcaux_lovaszloss_ft_lr0001_dict.model'))
model.load_state_dict(weight)
model = model.to(device)
model.eval()

Loaded pretrained resnet weights


UNetRes34HcAux(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running

In [7]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict(model, x, device=device, with_tta=True)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res34_hcaux_lovaszloss_ft_lr0001_tta_submission.csv')

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


# Model2 UNetRes34HcAuxSCSE lovaszloss ft

In [8]:
gc.collect()
torch.cuda.empty_cache()

model = M2.UNetRes34HcAuxSCSE(n_classes=1)
weight = torch.load(str(model_dir / 'unet_res34_hcaux_lovaszloss_ft_scse_dict.model'))
model.load_state_dict(weight)
model = model.to(device)
model.eval()

Loaded pretrained resnet weights


UNetRes34HcAuxSCSE(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_run

In [9]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict(model, x, device=device, with_tta=True)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res34_hcaux_scse_lovaszloss_ft_tta_submission.csv')

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


# Model2 UNetRes34HcAuxSCSEv2

In [6]:
gc.collect()
torch.cuda.empty_cache()

model = M2.UNetRes34HcAuxSCSEv2(n_classes=1)
weight = torch.load(str(model_dir / 'unet_res34_hcauxscsev2_dict.model'))
model.load_state_dict(weight)
model = model.to(device)
model.eval()

Loaded pretrained resnet weights


UNetRes34HcAuxSCSEv2(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_r

In [7]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict(model, x, device=device, with_tta=True)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res34_hcauxscsev2_tta_submission.csv')

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


# Model3 UNetRes34AuxSCSE CV0

In [6]:
gc.collect()
torch.cuda.empty_cache()

model = M3.UNetRes34AuxSCSE()
weight = torch.load(str(model_dir / 'unet_res34_auxscse_aug_kfold_cv0_phase2_dict.model'))
model.load_state_dict(weight)
model = model.to(device)
model.eval()

Loaded pretrained resnet weights


UNetRes34AuxSCSE(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

In [7]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict(model, x, device=device, use_sigmoid=False, threshold=0.0, with_tta=True)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res34_auxscse_cv0_tta_fixed_submission.csv')

# Model3 UNetRes34AuxSCSE CV0-3 Avg

In [6]:
gc.collect()
torch.cuda.empty_cache()

models = []
for i in range(3):
    model = M3.UNetRes34AuxSCSE()
    modelname = f'unet_res34_auxscse_aug_kfold_cv{i}_phase2_dict.model'
    weight = torch.load(str(model_dir / modelname))
    model.load_state_dict(weight)
    model = model.to(device)
    model.eval()
    models.append(model)
    print('Loaded: {}'.format(modelname))

Loaded pretrained resnet weights
Loaded: unet_res34_auxscse_aug_kfold_cv0_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_auxscse_aug_kfold_cv1_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_auxscse_aug_kfold_cv2_phase2_dict.model


In [7]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict_kfold_cv(models, x, device=device, use_sigmoid=False, threshold=0.0, with_tta=True)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res34_auxscse_cv0_1_2_avg_tta_submission.csv')

# Model3 UNetRes34AuxSCSE CV0-5 Avg

In [6]:
gc.collect()
torch.cuda.empty_cache()

models = []
for i in range(5):
    model = M3.UNetRes34AuxSCSE()
    modelname = f'unet_res34_auxscse_aug_kfold_cv{i}_phase2_dict.model'
    weight = torch.load(str(model_dir / modelname))
    model.load_state_dict(weight)
    model = model.to(device)
    model.eval()
    models.append(model)
    print('Loaded: {}'.format(modelname))

Loaded pretrained resnet weights
Loaded: unet_res34_auxscse_aug_kfold_cv0_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_auxscse_aug_kfold_cv1_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_auxscse_aug_kfold_cv2_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_auxscse_aug_kfold_cv3_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_auxscse_aug_kfold_cv4_phase2_dict.model


In [7]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict_kfold_cv(models, x, device=device, use_sigmoid=False, threshold=0.0, with_tta=True)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res34_auxscse_cv0_1_2_3_4_avg_tta_submission.csv')

# Model3 UNetRes34AuxSCSE CV0-5 Avg without TTA

In [6]:
gc.collect()
torch.cuda.empty_cache()

models = []
for i in range(5):
    model = M3.UNetRes34AuxSCSE()
    modelname = f'unet_res34_auxscse_aug_kfold_cv{i}_phase2_dict.model'
    weight = torch.load(str(model_dir / modelname))
    model.load_state_dict(weight)
    model = model.to(device)
    model.eval()
    models.append(model)
    print('Loaded: {}'.format(modelname))

Loaded pretrained resnet weights
Loaded: unet_res34_auxscse_aug_kfold_cv0_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_auxscse_aug_kfold_cv1_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_auxscse_aug_kfold_cv2_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_auxscse_aug_kfold_cv3_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_auxscse_aug_kfold_cv4_phase2_dict.model


In [7]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict_kfold_cv(models, x, device=device, use_sigmoid=False, threshold=0.0, with_tta=False)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res34_auxscse_cv0_1_2_3_4_avg_submission.csv')

# Model2 UNetRes34HcAuxSCSE CV0

In [6]:
gc.collect()
torch.cuda.empty_cache()

model = M2.UNetRes34HcAuxSCSEv2(n_classes=1, n_aux_classes=8)
modelname = 'unet_res34_hcauxscsev2_aug_kfold_cv0_phase2_dict.model'
weight = torch.load(str(model_dir / modelname))
model.load_state_dict(weight)
model = model.to(device)
model.eval()
print('Loaded: {}'.format(modelname))

Loaded pretrained resnet weights
Loaded: unet_res34_hcauxscsev2_aug_kfold_cv0_phase2_dict.model


In [7]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict(model, x, device=device, use_sigmoid=False, threshold=0.0, with_tta=True)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res34_hcauxscsev2_cv0_tta_submission.csv')

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


# Model3 UNetRes34AuxSimple CV0

In [6]:
gc.collect()
torch.cuda.empty_cache()

model = M3.UNetRes34AuxSimple(n_classes=1, n_aux_classes=8)
modelname = 'unet_res34_aux_simple_aug_kfold_cv0_phase2_dict.model'
weight = torch.load(str(model_dir / modelname))
model.load_state_dict(weight)
model = model.to(device)
model.eval()
print('Loaded: {}'.format(modelname))

Loaded pretrained resnet weights
Loaded: unet_res34_aux_simple_aug_kfold_cv0_phase2_dict.model


In [7]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict(model, x, device=device, use_sigmoid=False, threshold=0.0, with_tta=True)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res34_aux_simple_cv0_tta_submission.csv')

# Model3 UNetRes34AuxSimple2 CV0-3

In [6]:
gc.collect()
torch.cuda.empty_cache()

models = []
for i in range(3):
    model = M3.UNetRes34AuxSimple2(n_classes=1, n_aux_classes=8)
    modelname = f'unet_res34_aux_simple2_aug_kfold_cv{i}_phase2_dict.model'
    weight = torch.load(str(model_dir / modelname))
    model.load_state_dict(weight)
    model = model.to(device)
    model.eval()
    models.append(model)
    print('Loaded: {}'.format(modelname))

Loaded pretrained resnet weights
Loaded: unet_res34_aux_simple2_aug_kfold_cv0_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_aux_simple2_aug_kfold_cv1_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_aux_simple2_aug_kfold_cv2_phase2_dict.model


In [7]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict_kfold_cv(models, x, device=device, use_sigmoid=False, threshold=0.0, with_tta=True)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res34_aux_simple2_cv0_1_2_avg_tta_submission.csv')

# Model3 UNetRes34HcAuxSCSEv4 CV0

In [6]:
gc.collect()
torch.cuda.empty_cache()

model = M3.UNetRes34HcAuxSCSEv4(n_classes=1, n_aux_classes=8)
modelname = 'unet_res34_hcauxscse_v4_aug_kfold_cv0_phase2_dict.model'
weight = torch.load(str(model_dir / modelname))
model.load_state_dict(weight)
model = model.to(device)
model.eval()
print('Loaded: {}'.format(modelname))

Loaded pretrained resnet weights
Loaded: unet_res34_hcauxscse_v4_aug_kfold_cv0_phase2_dict.model


In [7]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict(model, x, device=device, use_sigmoid=False, threshold=0.0, with_tta=True)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res34_hcauxscse_v4_cv0_tta_submission.csv')

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


# Model3 UNetRes34HcAuxSCSEv4 CV0-3

In [6]:
gc.collect()
torch.cuda.empty_cache()

models = []
for i in range(3):
    model = M3.UNetRes34HcAuxSCSEv4(n_classes=1, n_aux_classes=8)
    modelname = f'unet_res34_hcauxscse_v4_aug_kfold_cv{i}_phase2_dict.model'
    weight = torch.load(str(model_dir / modelname))
    model.load_state_dict(weight)
    model = model.to(device)
    model.eval()
    models.append(model)
    print('Loaded: {}'.format(modelname))

Loaded pretrained resnet weights
Loaded: unet_res34_hcauxscse_v4_aug_kfold_cv0_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_hcauxscse_v4_aug_kfold_cv1_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_hcauxscse_v4_aug_kfold_cv2_phase2_dict.model


In [7]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict_kfold_cv(models, x, device=device, use_sigmoid=False, threshold=0.0, with_tta=True)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res34_hcauxscse_v4_cv0_1_2_avg_tta_submission.csv')

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


# Model3 UNetRes34HcAuxSCSEv4 AuxSimple2 CV0-3 Ensemble 

In [6]:
gc.collect()
torch.cuda.empty_cache()

models = []
for i in range(3):
    model = M3.UNetRes34AuxSimple2(n_classes=1, n_aux_classes=8)
    modelname = f'unet_res34_aux_simple2_aug_kfold_cv{i}_phase2_dict.model'
    weight = torch.load(str(model_dir / modelname))
    model.load_state_dict(weight)
    model = model.to(device)
    model.eval()
    models.append(model)
    print('Loaded: {}'.format(modelname))
    
for i in range(3):
    model = M3.UNetRes34HcAuxSCSEv4(n_classes=1, n_aux_classes=8)
    modelname = f'unet_res34_hcauxscse_v4_aug_kfold_cv{i}_phase2_dict.model'
    weight = torch.load(str(model_dir / modelname))
    model.load_state_dict(weight)
    model = model.to(device)
    model.eval()
    models.append(model)
    print('Loaded: {}'.format(modelname))

Loaded pretrained resnet weights
Loaded: unet_res34_aux_simple2_aug_kfold_cv0_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_aux_simple2_aug_kfold_cv1_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_aux_simple2_aug_kfold_cv2_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_hcauxscse_v4_aug_kfold_cv0_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_hcauxscse_v4_aug_kfold_cv1_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_hcauxscse_v4_aug_kfold_cv2_phase2_dict.model


In [7]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict_kfold_cv(models, x, device=device, use_sigmoid=False, threshold=0.0, with_tta=True)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_ensemble_simple2_and_v4_cv0_1_2_avg_tta_submission.csv')

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


# Model4 UNetRes34BilinearHcSCSEv5 CV5

In [8]:
gc.collect()
torch.cuda.empty_cache()

model = M4.UNetRes34BilinearHcSCSEv5(n_classes=1)
modelname = 'unet_res34_bilinear_hcscse_v5_kfold_cv4_phase2_dict.model'
weight = torch.load(str(model_dir / modelname))
model.load_state_dict(weight)
model = model.to(device)
model.eval()
print('Loaded: {}'.format(modelname))

Loaded pretrained resnet weights
Loaded: unet_res34_bilinear_hcscse_v5_kfold_cv4_phase2_dict.model


In [9]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict(model, x, device=device, use_sigmoid=False, threshold=0.0, with_tta=True)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res34_v5_best_singlemodel_cv5_tta_submission.csv')

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


# Model4 UNetRes34BilinearHcSCSEv5 CV4-5

In [6]:
gc.collect()
torch.cuda.empty_cache()

models = []
for i in (3, 4):
    model = M4.UNetRes34BilinearHcSCSEv5(n_classes=1)
    modelname = f'unet_res34_bilinear_hcscse_v5_kfold_cv{i}_phase2_dict.model'
    weight = torch.load(str(model_dir / modelname))
    model.load_state_dict(weight)
    model = model.to(device)
    model.eval()
    models.append(model)
    print('Loaded: {}'.format(modelname))

Loaded pretrained resnet weights
Loaded: unet_res34_bilinear_hcscse_v5_kfold_cv3_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_bilinear_hcscse_v5_kfold_cv4_phase2_dict.model


In [7]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict_kfold_cv(models, x, device=device, use_sigmoid=False, threshold=0.0, with_tta=True)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res34_v5_cv3_4_avg_tta_submission.csv')

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


# Model4 v5 Final CV0-4

In [6]:
gc.collect()
torch.cuda.empty_cache()

names = [
    'unet_res34_bilinear_hcscse_v5_kfold_cv0_phase2_dict.model',
    'unet_res34_bilinear_hcscse_v5_kfold_cv2_phase2_dict.model',
    'unet_res34_bilinear_hcscse_v5_kfold_cv3_phase2_dict.model',
    'unet_res34_bilinear_hcscse_v5_kfold_cv4_phase2_dict.model'
]

models = []
for modelname in names:
    model = M4.UNetRes34BilinearHcSCSEv5(n_classes=1)
    weight = torch.load(str(model_dir / modelname))
    model.load_state_dict(weight)
    model = model.to(device)
    model.eval()
    models.append(model)
    print('Loaded: {}'.format(modelname))
    
model = M3.UNetRes34HcAuxSCSEv4(n_classes=1, n_aux_classes=8)
modelname = 'unet_res34_hcauxscse_v4_aug_kfold_cv1_phase2_dict.model'
weight = torch.load(str(model_dir / modelname))
model.load_state_dict(weight)
model = model.to(device)
model.eval()
models.append(model)
print('Loaded: {}'.format(modelname))

Loaded pretrained resnet weights
Loaded: unet_res34_bilinear_hcscse_v5_kfold_cv0_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_bilinear_hcscse_v5_kfold_cv2_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_bilinear_hcscse_v5_kfold_cv3_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_bilinear_hcscse_v5_kfold_cv4_phase2_dict.model
Loaded pretrained resnet weights
Loaded: unet_res34_hcauxscse_v4_aug_kfold_cv1_phase2_dict.model


In [7]:
rles = []

with torch.no_grad():
    for x in tqdm_notebook(test_loader):
        pred = predict_kfold_cv(models, x, device=device, use_sigmoid=False, threshold=0.0, with_tta=True)
        rles += [rle_encode(p.numpy()) for p in pred]
        
test_df['rle_mask'] = rles
submit(test_df, 'unet_res34_v5_final_cv0_4_avg_tta_submission.csv')

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
